In [22]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [23]:
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\heart.csv')
df_XGB = df.copy()
df_RF = df.copy()
df_GNB = df.copy()

# XGBoost

In [24]:
train_X, test_X, train_y, test_y = train_test_split(df_XGB.drop('output', axis=1), df_XGB['output'], random_state=42)

XGBModel = XGBClassifier()

XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

(0.8147826086956522, 0.03668394315255914)

In [25]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8157894736842105

# XGB TUNING

In [26]:
XGBParam1 = {
    'max_depth': [2,4,6,8,10],
    'min_child_weight': [2,4,6,8]
}

XGB_Grid1 = GridSearchCV(XGBClassifier(verbosity=0,
                                       gamma=0,
                                       subsample=0.6,
                                       learning_rate=0.1,
                                       n_estimators=200), XGBParam1)
XGB_Grid1.fit(train_X, train_y)
print(XGB_Grid1.best_params_, XGB_Grid1.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

{'max_depth': 2, 'min_child_weight': 4} 0.8325603864734299


In [27]:
XGBParam2 = {
    'gamma':[0.0,0.1,0.2,0.3,0.4,0.5]
}

XGB_Grid2 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=2,
                                       min_child_weight=4,
                                       subsample=0.6,
                                       learning_rate=0.1,
                                       n_estimators=200), XGBParam2)
XGB_Grid2.fit(train_X, train_y)
print(XGB_Grid2.best_params_, XGB_Grid2.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

{'gamma': 0.2} 0.8413526570048309


C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [28]:
XGBParam3 = {
    'subsample':[0.5,0.6,0.7]
}

XGB_Grid3 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=2,
                                       min_child_weight=4,
                                       gamma=0.2,
                                       learning_rate=0.04,
                                       n_estimators=200), XGBParam3)
XGB_Grid3.fit(train_X, train_y)
print(XGB_Grid3.best_params_, XGB_Grid3.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

{'subsample': 0.6} 0.8457971014492754


C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [29]:
XGBParam4 = {
    'learning_rate':[0.03,0.04,0.06]
}

XGB_Grid4 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=2,
                                       min_child_weight=4,
                                       gamma=0.2,
                                       subsample=0.6,
                                       n_estimators=100), XGBParam4)
XGB_Grid4.fit(train_X, train_y)
print(XGB_Grid4.best_params_, XGB_Grid4.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

{'learning_rate': 0.04} 0.8545893719806763


In [30]:
XGBParam5 = {
    'n_estimators':[60, 80, 100]
}

XGB_Grid5 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=2,
                                       min_child_weight=4,
                                       gamma=0.2,
                                       subsample=0.6,
                                       learning_rate=0.04), XGBParam5)
XGB_Grid5.fit(train_X, train_y)
print(XGB_Grid5.best_params_, XGB_Grid5.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

{'n_estimators': 100} 0.8545893719806763


C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index,

In [31]:
XGBModel = XGBClassifier(verbosity=0,
                         max_depth=2,
                         min_child_weight=4,
                         gamma=0.2,
                         subsample=0.6,
                         learning_rate=0.04,
                         n_estimators=100)
XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

(0.8545893719806763, 0.033346633207513764)

In [32]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8421052631578947

# RandomForestClassifier

In [33]:
df_RF.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [34]:
train_X, test_X, train_y, test_y = train_test_split(df_RF.drop('output', axis=1), df_RF['output'], random_state=42)

RFModel = RandomForestClassifier()

RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

(0.8147826086956522, 0.031106910029313393)

In [35]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

0.8289473684210527

In [36]:
RFParam = {
           "max_depth":[7.5, 10, 12.5, 15],
           "n_estimators":[100, 150, 200],
            "max_samples": [.1,.2,.3]
        }

RF_Grid = GridSearchCV(RandomForestClassifier(), RFParam)
RF_Grid.fit(train_X, train_y)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [7.5, 10, 12.5, 15],
                         'max_samples': [0.1, 0.2, 0.3],
                         'n_estimators': [100, 150, 200]})

In [37]:
grid_results = pd.DataFrame(RF_Grid.cv_results_)
grid_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_samples,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.060118,0.002035,0.006034,0.000658,7.5,0.1,100,"{'max_depth': 7.5, 'max_samples': 0.1, 'n_esti...",0.847826,0.826087,0.755556,0.888889,0.844444,0.832560,0.043632,4
1,0.093987,0.003658,0.009971,0.002533,7.5,0.1,150,"{'max_depth': 7.5, 'max_samples': 0.1, 'n_esti...",0.847826,0.826087,0.733333,0.866667,0.755556,0.805894,0.052265,34
2,0.119901,0.005846,0.010987,0.000047,7.5,0.1,200,"{'max_depth': 7.5, 'max_samples': 0.1, 'n_esti...",0.869565,0.826087,0.733333,0.866667,0.800000,0.819130,0.050157,19
3,0.062201,0.001403,0.005958,0.000058,7.5,0.2,100,"{'max_depth': 7.5, 'max_samples': 0.2, 'n_esti...",0.847826,0.804348,0.755556,0.822222,0.822222,0.810435,0.030740,31
4,0.087174,0.001580,0.008299,0.000363,7.5,0.2,150,"{'max_depth': 7.5, 'max_samples': 0.2, 'n_esti...",0.847826,0.826087,0.800000,0.888889,0.800000,0.832560,0.033371,4
5,0.115968,0.001248,0.010807,0.000424,7.5,0.2,200,"{'max_depth': 7.5, 'max_samples': 0.2, 'n_esti...",0.869565,0.847826,0.777778,0.866667,0.822222,0.836812,0.034010,2
6,0.058983,0.001519,0.005810,0.000411,7.5,0.3,100,"{'max_depth': 7.5, 'max_samples': 0.3, 'n_esti...",0.826087,0.869565,0.777778,0.844444,0.822222,0.828019,0.030191,8
7,0.087067,0.001096,0.008166,0.000352,7.5,0.3,150,"{'max_depth': 7.5, 'max_samples': 0.3, 'n_esti...",0.826087,0.869565,0.755556,0.866667,0.755556,0.814686,0.050669,27
8,0.119568,0.001655,0.011238,0.000423,7.5,0.3,200,"{'max_depth': 7.5, 'max_samples': 0.3, 'n_esti...",0.847826,0.869565,0.755556,0.888889,0.800000,0.832367,0.048508,6
9,0.057336,0.001042,0.005611,0.000501,10,0.1,100,"{'max_depth': 10, 'max_samples': 0.1, 'n_estim...",0.847826,0.847826,0.777778,0.866667,0.777778,0.823575,0.038021,16


In [38]:
RFModel = RandomForestClassifier(max_depth=12.5, max_samples=0.1, n_estimators=100)

In [39]:
RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

(0.8191304347826087, 0.04605119182675276)

In [40]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

0.8552631578947368

# GaussianNB

In [41]:
df_GNB.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [42]:
train_X, test_X, train_y, test_y = train_test_split(df_GNB.drop('output', axis=1), df_RF['output'], random_state=42)

GNBModel = GaussianNB()

GNB_scores = cross_val_score(GNBModel, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

(0.783864734299517, 0.04793439159365486)

In [43]:
GNBModel.fit(train_X, train_y)
pred = GNBModel.predict(test_X)
accuracy_score(test_y, pred)

0.881578947368421

In [44]:
GNBParam = {
    'var_smoothing': [1e-9, 1e-6, 1e-5]
}

GNB_Grid = GridSearchCV(GaussianNB(), GNBParam)
GNB_Grid.fit(train_X, train_y)

GridSearchCV(estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 1e-06, 1e-05]})

In [45]:
grid_results = pd.DataFrame(GNB_Grid.cv_results_)
grid_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000549,0.000460,0.001550,0.002512,0.0,{'var_smoothing': 1e-09},0.76087,0.869565,0.755556,0.8,0.733333,0.783865,0.047934,2
1,0.001094,0.001754,0.000233,0.000467,0.000001,{'var_smoothing': 1e-06},0.76087,0.869565,0.755556,0.8,0.733333,0.783865,0.047934,2
2,0.002174,0.004348,0.000000,0.000000,0.00001,{'var_smoothing': 1e-05},0.76087,0.847826,0.755556,0.8,0.755556,0.783961,0.036008,1


In [46]:
GNBModel = GaussianNB(var_smoothing=1e-5)

In [47]:
GNB_scores = cross_val_score(GNBModel, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

(0.7839613526570048, 0.03600819840212659)

In [48]:
GNB_Grid.fit(train_X, train_y)
pred = GNB_Grid.predict(test_X)
accuracy_score(test_y, pred)

0.8947368421052632